In [22]:
import numpy as np
import math
from sympy import symbols, solve

import cv2 as cv
import matplotlib.pyplot as plt

# Singular Value Decomposition 

## Determinant

In [23]:
def getMinorEntry(matrix, row, col):
    n = matrix.shape[0]
    minor = np.zeros((n-1, n-1), dtype=object)
    rowIdx = 0
    for i in range(n):
        colIdx = 0
        if (i != row):
            for j in range(n):
                if (j != col):
                    minor[rowIdx, colIdx] = matrix[i, j]
                    colIdx += 1
            rowIdx += 1
    
    return getDeterminant(minor)

def getDeterminant(matrix):
    if (matrix.shape[0] == 1):
        return matrix[0, 0]
    elif (matrix.shape[0] == 2):
        return(matrix[0, 0]*matrix[1, 1] - (matrix[0, 1]*matrix[1, 0]))
    else:
        determinant = 0
        sign = 1
        for j in range(matrix.shape[1]):
            determinant += matrix[0, j]*getMinorEntry(matrix, 0, j)*sign
            sign *= -1;
    return determinant

## Gauss-Jordan Matrix Manipulation

In [24]:
def getIdxUtama(matrix, row):
    col = 0
    while (col < matrix.shape[1]):
        if (matrix[row, col] != 0):
            return col
        else:
            col += 1
    return -1

def swapRow(matrix, row1, row2):
    for j in range(matrix.shape[1]):
        tmp = matrix[row1, j]
        matrix[row1, j] = matrix[row2, j]
        matrix[row2, j] = tmp

def gaussElimination(matrix):
    mtemp = matrix.copy()
    row = mtemp.shape[0]
    col = mtemp.shape[1]
    zeroRow = row - 1
    for k in range(row):
        minIdx = k
        for i in range(k, row):
            if (getIdxUtama(mtemp, i) == -1):
                minIdx = i
            elif (getIdxUtama(mtemp, i) < getIdxUtama(mtemp, minIdx)):
                minIdx = i
        swapRow(mtemp, k, minIdx)

        if (getIdxUtama(mtemp, k) == -1):
            swapRow(mtemp, k, zeroRow)
            zeroRow -= 1

        idxUtama = getIdxUtama(mtemp, k)
        if (idxUtama != -1):
            divisor = mtemp[k, idxUtama]
            for j in range(col):
                mtemp[k, j] = mtemp[k, j] / divisor

        for i in range(k + 1, row):
            mult = mtemp[i, idxUtama]
            for j in range(col):
                mtemp[i, j] -= mult * mtemp[k, j]
    
    return mtemp

def gaussJordanElimination(matrix):
    mtemp = gaussElimination(matrix)
    row = mtemp.shape[0]
    col = mtemp.shape[1]
    for k in range(row - 1, -1, -1):
        idxUtama = getIdxUtama(mtemp, k)
        
        for i in range(k - 1, -1, -1):
            mult = mtemp[i, idxUtama]
            for j in range(col):
                mtemp[i, j] -= mult * mtemp[k, j]

    return mtemp

## Eigen-related Functions

In [25]:
def getEigenVal(matrix):
    mcopy = matrix.copy()
    x = symbols('x')
    nshape = mcopy.shape[0]
    lambdaId = np.eye(nshape)*x - mcopy

    expr = getDeterminant(lambdaId)
    val = solve(expr)

    # Ambil value unik saja
    val = [np.float(x) for x in set(val)]
    val.sort(reverse=True)
    return val

def getEigenMatrix(smat):
    eigenList = getEigenVal(smat)
    resultMatrix = []
    for ctr in range(len(eigenList)):
        eigval = eigenList[ctr]
        mdiag = np.eye(smat.shape[0])*eigval
        mtemp = mdiag - smat
        mtemp = gaussJordanElimination(mtemp)

        eigenVec = [0 for i in range(mtemp.shape[0])]
        delta = 0
        for i in range(len(eigenVec) - 1, -1 , -1):
            idxUtama = getIdxUtama(mtemp, i)
            
            # Kalo terlompat, diisi 1
            # Misal tadi isi x3 (idx 2) lalu x1 (idx 0) -> x2 kosong jadi assign 1
            if (i != len(eigenVec) - 1):
                delta = prevFilled - idxUtama - 1
                if (delta > 0):
                    while (delta != 0):
                        eigenVec[prevFilled - delta] = 1.0
                        delta -= 1

            if (idxUtama == -1):
                eigenVec[i] = 1.0
                prevFilled = i
            else:
                eigenVec[idxUtama] = 0
                prevFilled = idxUtama
                for j in range(mtemp.shape[1]):
                    if (j != idxUtama):
                        eigenVec[idxUtama] -= mtemp[idxUtama, j] * eigenVec[j]

        # Normalize vectors
        tmpList = [math.pow(x, 2) for x in eigenVec]
        vectorLength = math.sqrt(sum(tmpList))
        eigenVec = [x/vectorLength for x in eigenVec]

        resultMatrix.append(eigenVec)

    resultMatrix = np.array(resultMatrix, dtype=float)

    return resultMatrix.T

# Golub - Reinsh (halfway tried)

In [ ]:
eps = 1.e-10  # assumes double precision
tol = 1.e-64/eps
e = [0 for _ in range(n)]

for i in range(m):
    for j in range(n):
        U[i, j] = A[i, j]
# Householder reduction to bidiagonal form
g = 0.0; x = 0.0
for i in range(n):
    e[i] = g; s = 0.0; l = i + 1
    for j in range(i, m):
        s = s + u[j, i]**2
        if s < tol: g = 0.0
        else:
            f = u[i, i]
            g = math.sqrt(s) if f < 0.0 else -math.sqrt(s)
            h = f * g - s
            u[i, i] = f - g
            for j in range(l, n):
                s = 0.0
                for k in range(i, m): s = s + u[k, i] * u[k, j]
                f = s / h 
                for k in range(i, m): u[k, j] = u[k, j] + f * u[k, i]

        q[i] = g; s = 0.0
        for j in range(l, n): s = s + u[i, j]**2
        if s < tol: g = 0.0
        else:
            f = u[i, i + 1]
            g = math.sqrt(s) if f < 0.0 else -math.sqrt(s)
            h = f * g - s 
            u[i, i + 1] = f - g 
            for j in range(l, n): e[j] = u[i, j]/h 
            for j in range(l, m):
                s = 0.0
                for k in range(l, m): s = s + u[j, k] * u[i, k]
                for k in range(l, n): u[j, k] = u[j, k] + s * e[k]
        y = abs(q[i]) + abs(e[i])
        if y > x: x = y
# Accumulation of right-hand transformations
for i in range(n - 1, -1, -1):
    if g != 0:
        h = u[i, i + 1] * g
        for j in range(l - 1, n): v[j, i] = u[i, j]/h
        for j in range(l - 1, n):
            s = 0
            for k in range(l - 1, n):
                s = s + u[i, k] * v[k, j]
            for k in range(l - 1, n):
                v[k, j] = v[k, j] + s * v[k, i]
    
    for j in range(l - 1, n):
        v[i, j] = v[j, i] = 0
    v[i, i] = 1.0
    g = e[i]; l = i 

# Accumulation of left-hand transformations
for i in range(n-1,-1,-1):
    l = i+1
    g = q[i]
    for j in range(l,n): u[i][j] = 0.0
    if g != 0.0:
        h = u[i][i]*g
        for j in range(l,n):
            s=0.0
            for k in range(l,m): s += (u[k][i]*u[k][j])
            f = s/h
            for k in range(i,m): u[k][j] += (f*u[k][i])
        for j in range(i,m): u[j][i] = u[j][i]/g
    else:
        for j in range(i,m): u[j][i] = 0.0
    u[i][i] += 1.0

# Householder (failed)

In [ ]:
def getHouseholder(matrix, i):
    look = matrix[i:, i:]
    ai = np.matrix(look[:, 0]).T
    nshape = len(ai)

    ei = np.matrix([1 if i == 0 else 0 for i in range(nshape)]).T
    v1 = ai + sign(ai[0, 0])*np.linalg.norm(ai)*ei

    H = np.eye(nshape) - 2*(v1 @ v1.T)/(v1.T @ v1)
    Hi = np.eye(matrix.shape[0]) 
    Hi[i:, i:] = H

    return Hi

# m = sright.copy()
# for i in range(m.shape[0] - 1):
#     H = getHouseholder(m, i)
#     # U = H @ U
#     m = H @ m

# leftmost_col = np.matrix(m[:, 0]).T
# rest_col = m[:, 1:].T

# for i in range(m.shape[0] - 2):
#     rest_col = np.matrix(getHouseholder(rest_col, i) @ rest_col)

In [ ]:
def getColGivenRotation(matrix, i, j):
    aij, bij = matrix[i-1, j], matrix[i, j]
    norm = getNorm([aij, bij])
    if norm != 0:
        cos0 = aij/norm
        sin0 = bij/norm 

    rot = np.matrix([[cos0, sin0], 
                    [-sin0, cos0]])

    Gi = np.eye(matrix.shape[0])
    Gi[i - 1: i + 1, i - 1: i + 1] = rot
    return Gi

In [26]:
mat = np.array([[3, 1, 1],
                [-1, 3, 1]], dtype=float)
# mat = np.array([[1, 1],
#                 [0, 1],
#                 [1, 0]], dtype=float)
mshape = mat.shape
mat

array([[ 3.,  1.,  1.],
       [-1.,  3.,  1.]])

In [27]:
# Matrix singular
sleft = np.matmul(mat, mat.T)
sright = np.matmul(mat.T, mat)
sright

array([[10.,  0.,  2.],
       [ 0., 10.,  4.],
       [ 2.,  4.,  2.]])

In [28]:
leftEigen = getEigenVal(sleft)
rightEigen = getEigenVal(sright)
# Concat
concat = leftEigen.copy()
concat.extend(rightEigen) 

# Get nonzero unique value only
sigmaval = [math.sqrt(x) for x in set(concat) if x != 0]
sigmaval.sort(reverse = True)

In [29]:
# U Matrix
UMat = getEigenMatrix(sleft)

# Sigma Matrix
matsig = np.zeros(mshape, dtype=float)
np.fill_diagonal(matsig, sigmaval)

# VTranspose Matix
VTMat = getEigenMatrix(sright).T

In [30]:
UMat @ matsig @ VTMat

array([[ 3.,  1.,  1.],
       [-1.,  3.,  1.]])

In [31]:
rank = 2
U = np.reshape(UMat[:, 0], (mat.shape[0], 1))
VT = np.reshape(VTMat[0, :], (1, (mat.shape[1])))
compressedMatrix = U * matsig[0, 0] @ VT
for i in range(1, rank - 1):
    U = np.reshape(UMat[:, i], (mat.shape[0], 1))
    VT = np.reshape(VTMat[i, :], (1, (mat.shape[1])))
    compressedMatrix += U * matsig[i, i] @ VT

compressedMatrix

array([[1., 2., 1.],
       [1., 2., 1.]])

In [32]:
img = cv.imread('lena.png')
img = cv.resize(img, (10, 10))
gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
matrix = np.asarray(gray_img)

In [33]:
np.linalg.eigvals(matrix)[:5]

array([582.22144002 +0.j        ,  38.91244495+69.68141477j,
        38.91244495-69.68141477j, 118.24871468 +6.44933765j,
       118.24871468 -6.44933765j])

In [35]:
mcopy = matrix.copy()
x = symbols('x')
nshape = mcopy.shape[0]
lambdaId = np.eye(nshape)*x - mcopy
for k in range(1, nshape):
    for i in range(k, nshape):
        p = lambdaId[i, k - 1]
        q = lambdaId[k - 1, k - 1]

        if (q != 0):
            lambdaId[i, :] = lambdaId[i, :] - (p / q) * (lambdaId[k - 1, :])